In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [0]:
%run "./configuration"

In [0]:
spark.read.json(f"{raw_folder_path}/movies.json").printSchema()

root
 |-- adult: boolean (nullable = true)
 |-- backdrop_path: string (nullable = true)
 |-- belongs_to_collection: struct (nullable = true)
 |    |-- backdrop_path: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- poster_path: string (nullable = true)
 |-- budget: long (nullable = true)
 |-- credits: struct (nullable = true)
 |    |-- cast: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- adult: boolean (nullable = true)
 |    |    |    |-- cast_id: long (nullable = true)
 |    |    |    |-- character: string (nullable = true)
 |    |    |    |-- credit_id: string (nullable = true)
 |    |    |    |-- gender: long (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- known_for_department: string (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- order: long (nullable = true)
 |    |    |    |-- original_nam

In [0]:
database = "movies"

[FileInfo(path='abfss://raw@chautnd2mvdl.dfs.core.windows.net/movies.json', name='movies.json', size=5811978, modificationTime=1726563434000),
 FileInfo(path='abfss://raw@chautnd2mvdl.dfs.core.windows.net/people/', name='people/', size=0, modificationTime=1726563125000)]

In [0]:
print("hi")

hi


## Create Delta Lake database

In [0]:
%sql
drop database if exists dtl_movies cascade;
create database if not exists dtl_movies;

In [0]:
movies_raw_df = spark.read.json(f"{raw_folder_path}/movies.json")

In [0]:
print(movies_raw_df.count())

200


## Movie Fact

In [0]:
movie_fact_df = movies_raw_df.select(
  F.col("id").alias("movie_id"),
  F.col("imdb_id").alias("imdb_movie_id"),
  F.col("ltb_film_slug").alias("letterboxd_movie_id"),
  F.col("budget").alias("movie_budget"),
  F.col("revenue").alias("movie_revenue"),
  F.col("filmstat_likes").alias("letterboxd_likes"),
  F.col("filmstat_watches").alias("letterboxd_watches"),
  F.col("filmstat_lists").alias("letterboxd_lists"),
  F.col("letterboxd_ag_rating.best_rating").alias("letterboxd_rating_best"),
  F.col("letterboxd_ag_rating.worst_rating").alias("letterboxd_rating_worst"),
  F.col("letterboxd_ag_rating.rating_value").alias("letterboxd_rating_value"),
  F.col("letterboxd_ag_rating.rating_count").alias("letterboxd_rating_count"),
  F.col("letterboxd_ag_rating.review_count").alias("letterboxd_review_count"),
  F.col("title").alias("movie_title"),
  F.col("original_title").alias("movie_original_title"),
  F.col("runtime").alias("movie_runtime"),
  F.col("release_date").alias("movie_release_date"),
  F.col("original_language").alias("movie_original_language"),
  F.col("overview").alias("movie_overview"),
  F.col("status").alias("movie_status"),
  F.col("tagline").alias("movie_tagline"),
  F.col("belongs_to_collection.id").alias("movie_collection_id"),
  F.col("belongs_to_collection.name").alias("movie_collection_name"),
)

In [0]:
def check_if_table_exists(database, table):
    return spark.catalog.tableExists(f"{database}.{table}");

In [0]:
from delta.tables import DeltaTable

def upsert_delta_table(df, database_name, table_name, pKeys, partitionKey):
    mergeConditions = " and ".join(f"target.{pKey} = source.{pKey}" for pKey in pKeys)
    if (check_if_table_exists(database, table_name)):
        table = DeltaTable.forName(spark, f"{database_name}.{table_name}")
        table.alias("target").merge(
            df.alias("source"), 
            mergeConditions
        ) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll()
    else:
        if (partitionKey == None):
            df.write \
            .mode("overwrite") \
            .format("delta") \
            .saveAsTable(f"{database_name}.{table_name}")
        else:
            df.write \
            .mode("overwrite") \
            .format("delta") \
            .partitionBy(partitionKey) \
            .saveAsTable(f"{database_name}.{table_name}")

In [0]:
upsert_delta_table(movie_fact_df, "dtl_movies", "movie_fact", "movie_id", None)

In [0]:
%sql
show tables in dtl_movies;

database,tableName,isTemporary
dtl_movies,movie_fact,false
,_sqldf,true


In [0]:
%sql
select * from dtl_movies.movie_fact

movie_id,imdb_movie_id,letterboxd_movie_id,movie_budget,movie_revenue,letterboxd_likes,letterboxd_watches,letterboxd_lists,letterboxd_rating_best,letterboxd_rating_worst,letterboxd_rating_value,letterboxd_rating_count,letterboxd_review_count,movie_title,movie_original_title,movie_runtime,movie_release_date,movie_original_language,movie_overview,movie_status,movie_tagline,movie_collection_id,movie_collection_name
533535,tt6263850,deadpool-wolverine,200000000,1304995644,"Liked by 487,074 members","Watched by 1,320,936 members","Appears in 216,400 lists",5,0,3.59,1174016,467139,Deadpool & Wolverine,Deadpool & Wolverine,128,2024-07-24,en,"A listless Wade Wilson toils away in civilian life with his days as the morally flexible mercenary, Deadpool, behind him. But when his homeworld faces an existential threat, Wade must reluctantly suit-up again with an even more reluctant Wolverine.",Released,Come together.,448150,Deadpool Collection
365177,tt4978420,borderlands-2024,115000000,30863794,"Liked by 5,240 members","Watched by 66,570 members","Appears in 21,229 lists",5,0,1.58,59192,30007,Borderlands,Borderlands,101,2024-08-07,en,"Returning to her home planet, an infamous bounty hunter forms an unexpected alliance with a team of unlikely heroes. Together, they battle monsters and dangerous bandits to protect a young girl who holds the key to unimaginable power.",Released,Chaos loves company.,null,null
1022789,tt22022452,inside-out-2-2024,200000000,1676778335,"Liked by 455,635 members","Watched by 1,424,801 members","Appears in 166,500 lists",5,0,3.68,1237644,432415,Inside Out 2,Inside Out 2,97,2024-06-11,en,"Teenager Riley's mind headquarters is undergoing a sudden demolition to make room for something entirely unexpected: new Emotions! Joy, Sadness, Anger, Fear and Disgust, who’ve long been running a successful operation by all accounts, aren’t sure how to feel when Anxiety shows up. And it looks like she’s not alone.",Released,Make room for new emotions.,1022790,Inside Out Collection
646097,tt11301886,rebel-ridge,0,0,"Liked by 27,624 members","Watched by 91,009 members","Appears in 16,720 lists",5,0,3.52,79002,25704,Rebel Ridge,Rebel Ridge,132,2024-08-27,en,A former Marine confronts corruption in a small town when local law enforcement unjustly seizes the bag of cash he needs to post his cousin's bail.,Released,Their laws. His rules.,null,null
519182,tt7510222,despicable-me-4,100000000,940356614,"Liked by 72,159 members","Watched by 359,499 members","Appears in 49,136 lists",5,0,2.75,309824,130330,Despicable Me 4,Despicable Me 4,94,2024-06-20,en,"Gru and Lucy and their girls—Margo, Edith and Agnes—welcome a new member to the Gru family, Gru Jr., who is intent on tormenting his dad. Gru also faces a new nemesis in Maxime Le Mal and his femme fatale girlfriend Valentina, forcing the family to go on the run.",Released,Things just got a little more despicable.,86066,Despicable Me Collection
573435,tt4919268,bad-boys-ride-or-die,100000000,402529640,"Liked by 41,333 members","Watched by 147,967 members","Appears in 33,952 lists",5,0,3.22,129345,45883,Bad Boys: Ride or Die,Bad Boys: Ride or Die,115,2024-06-05,en,"After their late former Captain is framed, Lowrey and Burnett try to clear his name, only to end up on the run themselves.",Released,Miami's finest are now its most wanted.,14890,Bad Boys Collection
970347,tt1121948,the-killer-2024,30000000,0,"Liked by 1,760 members","Watched by 9,818 members","Appears in 3,060 lists",5,0,2.81,8172,3226,The Killer,The Killer,126,2024-08-22,en,"Zee is a feared contract killer known as ""the Queen of the Dead,"" but when she refuses to murder a young blind woman, she finds herself hunted both by criminal colleagues and a determined police detective.",Released,Some alliances form. Others will crumble.,null,null
718821,tt12584954,twisters,155000000,364092150,"Liked by 179,292 members","Watched by 585,742 members","Appears in 98,987 lists",5,0,3.36,519878,245120,Twisters,Twisters,123,2024-07-10,en,"As s

## Credits Dimension

In [0]:
credits_df = movies_raw_df.select("id", "credits")

In [0]:
crew_df = credits_df.select(F.col("id").alias("movie_id"), F.explode("credits.crew").alias("crew_member"))
cast_df = credits_df.select(F.col("id").alias("movie_id"), F.explode("credits.cast").alias("cast_member"))

In [0]:
movie_crew_df = crew_df.select("movie_id", F.col("crew_member.id").alias("person_id"), F.col("crew_member.department").alias("department"), F.col("crew_member.job").alias("job")) \
    .withColumn("character", F.lit(None)) \
    .withColumn("role", F.lit("crew"))
movie_cast_df = cast_df.select("movie_id", F.col("cast_member.id").alias("person_id"), F.col("cast_member.character").alias("character")) \
    .withColumn("department", F.lit(None)) \
    .withColumn("job", F.lit(None)) \
    .withColumn("role", F.lit("cast"))

In [0]:
movie_person_bridge = movie_crew_df.unionByName(movie_cast_df)

In [0]:
movie_person_bridge_with_order = movie_person_bridge \
    .withColumn("order", F.row_number().over(Window.partitionBy("person_id", "movie_id").orderBy("role")))

In [0]:
# person who participate in both crew and cast in the same movie
movie_person_bridge_with_order.filter("person_id = 10859").display()

movie_id,person_id,department,job,character,role,order
533535,10859,null,null,Wade Wilson / Deadpool / Nicepool,cast,1
533535,10859,Production,Producer,null,crew,2
533535,10859,Writing,Writer,null,crew,3
639720,10859,null,null,Cal,cast,1
639720,10859,Production,Producer,null,crew,2


In [0]:
# check people who participate in both crew and cast in the same movie
selected_movie_person = movie_person_bridge_with_order.select("person_id", "movie_id", "role")
condition = (F.col("t1.person_id") == F.col("t2.person_id")) & \
            (F.col("t1.movie_id") == F.col("t2.movie_id")) & \
            (F.col("t1.role") != F.col("t2.role"))

selected_movie_person.alias("t1") \
    .join(selected_movie_person.alias("t2"), condition, "inner") \
    .display()

person_id,movie_id,role,person_id,movie_id,role
10859,533535,crew,10859,533535,cast
10859,533535,crew,10859,533535,cast
4733517,533535,crew,4733517,533535,cast
4857614,533535,crew,4857614,533535,cast
2280082,533535,crew,2280082,533535,cast
4115448,533535,crew,4115448,533535,cast
2423284,533535,crew,2423284,533535,cast
2423287,533535,crew,2423287,533535,cast
4858628,533535,crew,4858628,533535,cast
2151833,533535,crew,2151833,533535,cast


In [0]:
crew_info_df = crew_df.select(
    F.col("crew_member.id").alias("person_id"),
    F.col("crew_member.known_for_department").alias("department"),
    F.col("crew_member.name").alias("name"),
    F.col("crew_member.original_name").alias("original_name"),
    F.col("crew_member.gender").alias("gender")
)
cast_info_df = cast_df.select(
    F.col("cast_member.id").alias("person_id"),
    F.col("cast_member.known_for_department").alias("department"),
    F.col("cast_member.name").alias("name"),
    F.col("cast_member.original_name").alias("original_name"),
    F.col("cast_member.gender").alias("gender")
)

In [0]:
person_df = crew_info_df.unionByName(cast_info_df)

In [0]:
person_final_df = person_df.dropDuplicates(["person_id"])

In [0]:
print(person_df.count())
print(person_final_df.count())

18463
16205


### Get additional information on crew and cast

In [0]:
person_info_df = spark.read.json(f"{raw_folder_path}/people/280180_credits.json")

In [0]:
ids = person_info_df.columns

In [0]:
ids

['1002466',
 '1082034',
 '1084758',
 '10952',
 '11455',
 '1158414',
 '1173014',
 '1211000',
 '1219572',
 '1236',
 '1256256',
 '1256475',
 '1268072',
 '1324652',
 '1335818',
 '133835',
 '1349483',
 '1358061',
 '1367105',
 '1435644',
 '1463131',
 '1465838',
 '1481139',
 '1483455',
 '1503731',
 '1512040',
 '1519576',
 '1527698',
 '1546115',
 '1550831',
 '1588581',
 '1590289',
 '1591489',
 '1602526',
 '1607512',
 '1610839',
 '1655675',
 '1660987',
 '1665179',
 '1690489',
 '1699143',
 '1732965',
 '1734674',
 '1765207',
 '1768008',
 '1771689',
 '1783270',
 '1784360',
 '1790952',
 '1815847',
 '1825887',
 '1852531',
 '1866449',
 '1914648',
 '19280',
 '201389',
 '2014405',
 '2082467',
 '228304',
 '2332587',
 '2351555',
 '24045',
 '2441587',
 '2446',
 '2446240',
 '2480707',
 '2541004',
 '2569003',
 '2619207',
 '2714815',
 '2731556',
 '2738268',
 '2889659',
 '2962798',
 '3024895',
 '3125907',
 '3327172',
 '3382560',
 '3466804',
 '3498713',
 '3530948',
 '3582782',
 '3600065',
 '3754488',
 '3832058

In [0]:
ids_unpack = ",".join(ids)

In [0]:
ids_unpack

'1002466,1082034,1084758,10952,11455,1158414,1173014,1211000,1219572,1236,1256256,1256475,1268072,1324652,1335818,133835,1349483,1358061,1367105,1435644,1463131,1465838,1481139,1483455,1503731,1512040,1519576,1527698,1546115,1550831,1588581,1590289,1591489,1602526,1607512,1610839,1655675,1660987,1665179,1690489,1699143,1732965,1734674,1765207,1768008,1771689,1783270,1784360,1790952,1815847,1825887,1852531,1866449,1914648,19280,201389,2014405,2082467,228304,2332587,2351555,24045,2441587,2446,2446240,2480707,2541004,2569003,2619207,2714815,2731556,2738268,2889659,2962798,3024895,3125907,3327172,3382560,3466804,3498713,3530948,3582782,3600065,3754488,3832058,3881469,3893463,4034833,4038683,4040792,40481,4156534,4189304,4443,466391,4689,4724,476163,4809376,4809377,4809379,4809395,4809408,4809410,4809412,4809413,4809414,4809415,4809417,4809419,4844836,4844840,4892698,5215,53254,56358,582924,71556,770,772,774,776,777,778,781,929145,98394'

In [0]:
person_info_df.select(ids).display()

1002466,1082034,1084758,10952,11455,1158414,1173014,1211000,1219572,1236,1256256,1256475,1268072,1324652,1335818,133835,1349483,1358061,1367105,1435644,1463131,1465838,1481139,1483455,1503731,1512040,1519576,1527698,1546115,1550831,1588581,1590289,1591489,1602526,1607512,1610839,1655675,1660987,1665179,1690489,1699143,1732965,1734674,1765207,1768008,1771689,1783270,1784360,1790952,1815847,1825887,1852531,1866449,1914648,19280,201389,2014405,2082467,228304,2332587,2351555,24045,2441587,2446,2446240,2480707,2541004,2569003,2619207,2714815,2731556,2738268,2889659,2962798,3024895,3125907,3327172,3382560,3466804,3498713,3530948,3582782,3600065,3754488,3832058,3881469,3893463,4034833,4038683,4040792,40481,4156534,4189304,4443,466391,4689,4724,476163,4809376,4809377,4809379,4809395,4809408,4809410,4809412,4809413,4809414,4809415,4809417,4809419,4844836,4844840,4892698,5215,53254,56358,582924,71556,770,772,774,776,777,778,781,929145,98394
"List(false, List(), , 1953-10-01, null, 2, null, 1002466, nm0024839, Acting, Robert Amico, Brooklyn, New York City, New York, USA, 0.782, /mxZUbE9G3afbrTT9QPKGWpQcyjJ.jpg)","List(false, List(Big Bad Joe, Sky Anderson, Joseph E. Davis), , 1958-02-19, null, 2, null, 1082034, nm0204852, Acting, Joe Davis, Ann Arbor, Michigan, USA, 2.098, /aRMjeQIXYrJlYhJpeNtzRZndry8.jpg)","List(false, List(), , 1972-08-02, null, 2, null, 1084758, nm2286416, Acting, James Preston Rogers, Toronto, Ontario, Canada, 7.222, /6eVau2kFnU15T7D35OCt1uk5cIl.jpg)","List(false, List(Lorenzo DiBonaventura, Lorenzo Di Bonaventura), Di Bonaventura is best known for being the President of Worldwide Production for Warner Bros. Pictures. Di Bonaventura had been an executive at Warner Bros. since the 1990s. During his tenure at Warner Bros. he has discovered and shepherded many blockbuster movies. Di Bonaventura is responsible bringing pop culture classics The Matrix and Harry Potter, among others to the big screen. Di Bonaventura also owns his own production company called Di Bonaventura Pictures and is based at Paramount Pictures. He is best known for producing the G.I. Joe and Transformers film series. The films he produced have earned over $7 billion at the box office., 1957-01-13, null, 2, null, 10952, nm0225146, Production, Lorenzo di Bonaventura, New York City, New York, USA, 3.091, /skMUk4eVV6e08mv31A48jdYpEPp.jpg)","List(false, List(), Dan Lebental is known for Iron Man (2008), Ant-Man (2015) and 11:14 (2003)., null, null, 2, null, 11455, nm0495603, Editing, Dan Lebental, USA, 2.661, null)","List(false, List(Eugene Samuel Kim), Eugene Kim is known for his work on Brooklyn Nine-Nine (2013), I Think You Should Leave with Tim Robinson (2019) and The Lincoln Lawyer (2022)., null, null, 2, null, 1158414, nm1603142, Acting, Eugene Kim, null, 4.232, /4gunuSd9AWiIVjLmUwKoePB0LtU.jpg)","List(false, List(), , null, null, 2, null, 1173014, nm2900814, Production, J.P. Riley, null, 0.659, null)","List(false, List(), , 1970-01-01, null, 2, null, 1211000, nm3230448, Writing, Tom Gormican, null, 4.312, /7XdjdjSdiCTSUp0f30fsTtLBXjS.jpg)","List(false, List(), , null, null, 2, null, 1219572, nm1072410, Writing, Kevin Etten, null, 1.435, null)","List(false, List(Henry Taggert), Mark Ross Pellegrino (born April 9, 1965) is an American actor of film and television. He is best known for his work on Lost and Supernatural., 1965-04-09, null, 2, null, 1236, nm0671032, Acting, Mark Pellegrino, Los Angeles, California, USA, 20.671, /aYlypQrn0LLd1KjHWZDBYcWRvTl.jpg)","List(false, List(), , 1952-05-12, null, 2, null, 1256256, nm1783154, Acting, Mobin Khan, Varanasi, Uttar Pradesh, India, 0.746, null)","List(false, List(), , null, null, 0, null, 1256475, nm1350260, Creator, Gregory Ecklund, null, 0.001, null)","List(false, List(), , null, null, 2, null, 1268072, nm4978849, Acting, Joseph Aviel, null, 2.436, /hYf3iMSrhlQ2nJepAKwsL1C4MoK.jpg)","List(false, List(Melinda Sue Gorden), , null, null, 1, null, 1324652, nm0330449, Camera, Melinda Sue Gordon, null, 1.1

In [0]:
exploded_df.display()

info
1002466
1082034
1084758
10952
11455
1158414
1173014
1211000
1219572
1236


In [0]:
person_info_df.display()

1002466,1082034,1084758,10952,11455,1158414,1173014,1211000,1219572,1236,1256256,1256475,1268072,1324652,1335818,133835,1349483,1358061,1367105,1435644,1463131,1465838,1481139,1483455,1503731,1512040,1519576,1527698,1546115,1550831,1588581,1590289,1591489,1602526,1607512,1610839,1655675,1660987,1665179,1690489,1699143,1732965,1734674,1765207,1768008,1771689,1783270,1784360,1790952,1815847,1825887,1852531,1866449,1914648,19280,201389,2014405,2082467,228304,2332587,2351555,24045,2441587,2446,2446240,2480707,2541004,2569003,2619207,2714815,2731556,2738268,2889659,2962798,3024895,3125907,3327172,3382560,3466804,3498713,3530948,3582782,3600065,3754488,3832058,3881469,3893463,4034833,4038683,4040792,40481,4156534,4189304,4443,466391,4689,4724,476163,4809376,4809377,4809379,4809395,4809408,4809410,4809412,4809413,4809414,4809415,4809417,4809419,4844836,4844840,4892698,5215,53254,56358,582924,71556,770,772,774,776,777,778,781,929145,98394
"List(false, List(), , 1953-10-01, null, 2, null, 1002466, nm0024839, Acting, Robert Amico, Brooklyn, New York City, New York, USA, 0.782, /mxZUbE9G3afbrTT9QPKGWpQcyjJ.jpg)","List(false, List(Big Bad Joe, Sky Anderson, Joseph E. Davis), , 1958-02-19, null, 2, null, 1082034, nm0204852, Acting, Joe Davis, Ann Arbor, Michigan, USA, 2.098, /aRMjeQIXYrJlYhJpeNtzRZndry8.jpg)","List(false, List(), , 1972-08-02, null, 2, null, 1084758, nm2286416, Acting, James Preston Rogers, Toronto, Ontario, Canada, 7.222, /6eVau2kFnU15T7D35OCt1uk5cIl.jpg)","List(false, List(Lorenzo DiBonaventura, Lorenzo Di Bonaventura), Di Bonaventura is best known for being the President of Worldwide Production for Warner Bros. Pictures. Di Bonaventura had been an executive at Warner Bros. since the 1990s. During his tenure at Warner Bros. he has discovered and shepherded many blockbuster movies. Di Bonaventura is responsible bringing pop culture classics The Matrix and Harry Potter, among others to the big screen. Di Bonaventura also owns his own production company called Di Bonaventura Pictures and is based at Paramount Pictures. He is best known for producing the G.I. Joe and Transformers film series. The films he produced have earned over $7 billion at the box office., 1957-01-13, null, 2, null, 10952, nm0225146, Production, Lorenzo di Bonaventura, New York City, New York, USA, 3.091, /skMUk4eVV6e08mv31A48jdYpEPp.jpg)","List(false, List(), Dan Lebental is known for Iron Man (2008), Ant-Man (2015) and 11:14 (2003)., null, null, 2, null, 11455, nm0495603, Editing, Dan Lebental, USA, 2.661, null)","List(false, List(Eugene Samuel Kim), Eugene Kim is known for his work on Brooklyn Nine-Nine (2013), I Think You Should Leave with Tim Robinson (2019) and The Lincoln Lawyer (2022)., null, null, 2, null, 1158414, nm1603142, Acting, Eugene Kim, null, 4.232, /4gunuSd9AWiIVjLmUwKoePB0LtU.jpg)","List(false, List(), , null, null, 2, null, 1173014, nm2900814, Production, J.P. Riley, null, 0.659, null)","List(false, List(), , 1970-01-01, null, 2, null, 1211000, nm3230448, Writing, Tom Gormican, null, 4.312, /7XdjdjSdiCTSUp0f30fsTtLBXjS.jpg)","List(false, List(), , null, null, 2, null, 1219572, nm1072410, Writing, Kevin Etten, null, 1.435, null)","List(false, List(Henry Taggert), Mark Ross Pellegrino (born April 9, 1965) is an American actor of film and television. He is best known for his work on Lost and Supernatural., 1965-04-09, null, 2, null, 1236, nm0671032, Acting, Mark Pellegrino, Los Angeles, California, USA, 20.671, /aYlypQrn0LLd1KjHWZDBYcWRvTl.jpg)","List(false, List(), , 1952-05-12, null, 2, null, 1256256, nm1783154, Acting, Mobin Khan, Varanasi, Uttar Pradesh, India, 0.746, null)","List(false, List(), , null, null, 0, null, 1256475, nm1350260, Creator, Gregory Ecklund, null, 0.001, null)","List(false, List(), , null, null, 2, null, 1268072, nm4978849, Acting, Joseph Aviel, null, 2.436, /hYf3iMSrhlQ2nJepAKwsL1C4MoK.jpg)","List(false, List(Melinda Sue Gorden), , null, null, 1, null, 1324652, nm0330449, Camera, Melinda Sue Gordon, null, 1.1

In [0]:
person_info_df.select("1002466.adult")

DataFrame[adult: boolean]

In [0]:
help(person_info_df.groupBy().pivot)

Help on method pivot in module pyspark.sql.group:

pivot(pivot_col: str, values: Optional[List[ForwardRef('LiteralType')]] = None) -> 'GroupedData' method of pyspark.sql.group.GroupedData instance
    Pivots a column of the current :class:`DataFrame` and performs the specified aggregation.
    
    .. versionadded:: 1.6.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    pivot_col : str
        Name of the column to pivot.
    values : list, optional
        List of values that will be translated to columns in the output DataFrame.
    
        If ``values`` is not provided, Spark will eagerly compute the distinct values in
        ``pivot_col`` so it can determine the resulting schema of the transformation. To avoid
        any eager computations, provide an explicit list of values.
    
    Examples
    --------
    >>> from pyspark.sql import Row
    >>> df1 = spark.createDataFrame([
    ...     Row(course="dotNET", year=2012, 

In [0]:
person_info_df.selectExp

In [0]:
person_info_df.put(10)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
help(person_info_df.melt)

Help on method melt in module pyspark.sql.dataframe:

melt(ids: Union[ForwardRef('ColumnOrName'), List[ForwardRef('ColumnOrName')], Tuple[ForwardRef('ColumnOrName'), ...]], values: Union[ForwardRef('ColumnOrName'), List[ForwardRef('ColumnOrName')], Tuple[ForwardRef('ColumnOrName'), ...], NoneType], variableColumnName: str, valueColumnName: str) -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Unpivot a DataFrame from wide format to long format, optionally leaving
    identifier columns set. This is the reverse to `groupBy(...).pivot(...).agg(...)`,
    except for the aggregation, which cannot be reversed.
    
    :func:`melt` is an alias for :func:`unpivot`.
    
    .. versionadded:: 3.4.0
    
    Parameters
    ----------
    ids : str, Column, tuple, list, optional
        Column(s) to use as identifiers. Can be a single column or column name,
        or a list or tuple for multiple columns.
    values : str, Column, tuple, list, optional
        Column(s) to 

## Production Company Dimension

In [0]:
production_companies_raw_df = movies_raw_df \
    .select("id", "production_companies")

In [0]:
production_companies_exploded_df = production_companies_raw_df.select(F.col("id").alias("movie_id"), F.explode("production_companies").alias("production_company"))

In [0]:
production_movie_bridge_df = production_companies_exploded_df \
    .select(F.col("movie_id"), F.col("production_company.id").alias("production_company_id"))

In [0]:
production_companies_df = production_companies_exploded_df.select(
    F.col("production_company.id").alias("production_company_id"),
    F.col("production_company.name").alias("production_company_name"),
    F.col("production_company.logo_path").alias("logo_path"),
    F.col("production_company.origin_country").alias("origin_country_id"))

In [0]:
production_countries_df = movies_raw_df.select(F.explode("production_countries").alias("production_country")).distinct()

In [0]:
production_companies_joined_df = production_companies_df.join(production_countries_df, production_companies_df["origin_country_id"] == production_countries_df["production_country.iso_3166_1"], "left")

In [0]:
production_companies_final_df = production_companies_joined_df.select("production_company_id", "logo_path", "production_company_name", "origin_country_id", F.col("production_country.name").alias("country_english_name"))

## Genre Dimension

In [0]:
genres_raw_df = movies_raw_df \
    .select(F.col("id").alias("movie_id"), "genres")

In [0]:
genres_exploded_df = genres_raw_df.select(F.col("movie_id"), F.explode("genres").alias("genre"))

In [0]:
genre_movie_bridge = genres_exploded_df.select("movie_id", F.col("genre.id").alias("genre_id"))

In [0]:
genre_final_df = genres_exploded_df \
    .select(F.col("genre.id").alias("genre_id"), F.col("genre.name").alias("genre_name")) \
    .dropDuplicates(["genre_id"])

## Keyword Dimension

In [0]:
keywords_raw_df = movies_raw_df \
    .select(F.col("id").alias("movie_id"), "keywords.keywords")

In [0]:
keywords_exploded_df = keywords_raw_df.select(F.col("movie_id"), F.explode("keywords").alias("keyword"))

In [0]:
keyword_movie_bridge = keywords_exploded_df.select("movie_id", F.col("keyword.id").alias("keyword_id"))

In [0]:
keyword_final_df = keywords_exploded_df \
    .select(F.col("keyword.id").alias("keyword_id"), F.col("keyword.name").alias("keyword_name")) \
    .dropDuplicates(["keyword_id"])

## Spoken Language Dimension

In [0]:
spoken_languages_raw_df = movies_raw_df.select(F.col("id").alias("movie_id"), F.col("spoken_languages"))

In [0]:
spoken_languages_exploded_df = spoken_languages_raw_df.select("movie_id", F.explode("spoken_languages").alias("spoken_language"))

In [0]:
spoken_language_movie_bridge_df = spoken_languages_exploded_df.select("movie_id", F.col("spoken_language.iso_639_1").alias("spoken_language_id"))

In [0]:
spoken_language_df = spoken_languages_exploded_df.select(
    F.col("spoken_language.iso_639_1").alias("iso_639_1"),
    F.col("spoken_language.name").alias("language_name"),
    F.col("spoken_language.english_name").alias("english_name")
)